In [1]:
# All imports
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [80]:
# Read all files in this cell
temp_raw = pd.read_csv('../../resources/dataset_weather/us_temperature.csv')
humidity_raw = pd.read_csv('../../resources/dataset_weather/us_humidity.csv')
wind_speed_raw = pd.read_csv('../../resources/dataset_weather/us_wind_speed.csv')
accidents_raw = pd.read_csv('../../resources/us_accidents.csv')

In [81]:
accidents_raw['Start_Time'] = pd.to_datetime(accidents_raw['Start_Time'])
accidents_raw['Date'] = accidents_raw['Start_Time'].dt.date
accidents = accidents_raw.drop(columns=['Start_Time','End_Time', 'Distance(mi)'])
display(accidents)

,Severity,City,Date
0,3,Dayton,2016-02-08
1,2,Reynoldsburg,2016-02-08
2,2,Williamsburg,2016-02-08
3,3,Dayton,2016-02-08
4,2,Dayton,2016-02-08
...,...,...,...
1128106,2,Los Angeles,2017-08-30
1128107,2,Los Angeles,2017-08-30
1128108,3,Redlands,2017-08-30
1128109,4,Chatsworth,2017-08-30


In [74]:
# Cel to filter dates to a range of 2016 and 2017

# First convert dates to DateTime format
temp_raw['datetime'] = pd.to_datetime(temp_raw['datetime'])
humidity_raw['datetime'] = pd.to_datetime(humidity_raw['datetime'])
wind_speed_raw['datetime'] = pd.to_datetime(wind_speed_raw['datetime'])

# Filter the dates using .loc()
temp = temp_raw.loc[(temp_raw['datetime'] >= '2016-01-01') & (temp_raw['datetime'] < '2018-01-01')]
humidity = humidity_raw.loc[(humidity_raw['datetime'] >= '2016-01-01') & (humidity_raw['datetime'] < '2018-01-01')]
wind_speed = wind_speed_raw.loc[(wind_speed_raw['datetime'] >= '2016-01-01') & (wind_speed_raw['datetime'] < '2018-01-01')]

# Reset indexes
temp = temp.reset_index(drop=True)
humidity = humidity.reset_index(drop=True)
wind_speed = wind_speed.reset_index(drop=True)

/var/folders/5r/4dlwcpf56d76tctrcxgmk99w0000gn/T/ipykernel_74778/3732266852.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  temp_raw['datetime'] = pd.to_datetime(temp_raw['datetime'])
/var/folders/5r/4dlwcpf56d76tctrcxgmk99w0000gn/T/ipykernel_74778/3732266852.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  humidity_raw['datetime'] = pd.to_datetime(humidity_raw['datetime'])
/var/folders/5r/4dlwcpf56d76tctrcxgmk99w0000gn/T/ipykernel_74778/3732266852.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  wind_speed_raw['datetime'] = pd.to_datetime(wind_speed_raw['datetime

In [75]:
# Melting for all the weather dataframes
temp_melted = pd.melt(temp, id_vars=['datetime'], var_name='State', value_name='Temperature')
humidity_melted = pd.melt(humidity, id_vars=['datetime'], var_name='State', value_name='Humidity')
wind_speed_melted = pd.melt(wind_speed, id_vars=['datetime'], var_name='State', value_name='Wind_Speed')

In [76]:
# Converting beaufort windspeed to km/h
beaufort_scale = {
    0: (0, 1),
    1: (1, 5),
    2: (6, 11),
    3: (12, 19),
    4: (20, 28),
    5: (29, 38),
    6: (39, 49),
    7: (50, 61),
    8: (62, 74),
    9: (75, 88),
    10: (89, 102),
    11: (103, 117),
    12: (118, 133),
    13: (134, 149),
    14: (150, 166),
    15: (167, 183),
    16: (184, 201),
    17: (202, 220)
}

# Calculate the mean wind speed for each Beaufort scale number
beaufort_mean_speeds = {b: (rng[0] + rng[1]) / 2 for b, rng in beaufort_scale.items()}

# Map the Beaufort scale values to mean km/h values
wind_speed_melted['Wind_Speed_km'] = wind_speed_melted['Wind_Speed'].map(beaufort_mean_speeds)
wind_speed_melted = wind_speed_melted.drop(columns=['Wind_Speed'])

In [77]:
# Merge all three weather dataframes with inner join
weather = pd.merge(temp_melted, humidity_melted,on=['datetime','State'], how='inner')
weather = pd.merge(weather, wind_speed_melted,on=['datetime','State'], how='inner')

In [78]:
# Convert datetime to date
weather['Date'] = weather['datetime'].dt.date
weather = weather.drop(columns=['datetime'])

# Group on day and state
weather = weather.groupby(['Date', 'State']).mean()
display(weather)


Temperature   Humidity  Wind_Speed_km
Date       State                                               
2016-01-01 Albuquerque     267.674555  67.208333      25.895833
           Atlanta         279.687402  80.166667      26.062500
           Boston          275.848301  63.291667      15.395833
           Charlotte       284.268656  71.958333       8.875000
           Denver          264.457188  62.416667       8.791667
...                               ...        ...            ...
2017-12-11 Pittsburgh      274.298750  52.250000       5.875000
           Portland        283.623333  86.041667      38.562500
           San Diego       287.846667  80.833333       9.104167
           San Francisco          NaN        NaN            NaN
           Seattle         283.255833  88.583333      11.729167

[13300 rows x 3 columns]